## PROYECTO SPRINT 10
---

### 1. Descripción del proyecto 
Se ha decidido abrir un pequeño café regentado por robots en Los Ángeles. El proyecto es prometedor pero caro, asi que se decide intentar atraer inversionistas que están interesados en las condiciones actuales del mercado.

A través de un estudio de mercado con los datos procedentes de fuentes abiertas sobre restaurantes en LA.

---
### 2. Carga de dataset y procesamiento de datos 

In [9]:
# Importación de librerías neceserarias
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

In [10]:
#Asignación de variable al dataset
rest_data = pd.read_csv('rest_data_us_upd.csv')

In [36]:
#Visualización de la información del dataset
rest_data.info()
rest_data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9648 non-null   object
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 452.5+ KB


,id,object_name,address,chain,object_type,number
7211,18997,DOMINOS PIZZA,3309 W PICO BLVD,True,Pizza,2
6879,18665,BEYLUL RESTAURANT,5477 W ADAMS BLVD,True,Restaurant,90
5751,17537,TEDDY'S BAR,7000 HOLLYWOOD BLVD,False,Bar,153
104,11890,BARNEY'S BEANERY,200 WORLD WAY 240,False,Bar,75
745,12531,LOS ANGELES NEW YORK PIZZA,518 W 6TH ST,False,Pizza,34
6830,18616,CRENSHAW LIVE SUPPER CLUB,3888 CRENSHAW BLVD,False,Restaurant,215
432,12218,W HOTEL LOS ANGELES/WESTWOOD,930 HILGARD AVE,False,Restaurant,49
5170,16956,SUGAR FACTORY CONCESSION & CANDY STORE,6925 hollywood BLVD,False,Restaurant,5
721,12507,TWO BOOTS PIZZA,828 S BROADWAY,False,Pizza,19
6860,18646,LITTLE ONGPIN FAST FOOD,1700 BEVERLY BLVD,True,Restaurant,26


Al analizar el dataset, tanto a través de la visualización de su información como de una muestra representativa, podemos identificar el tipo de datos de cada columna y detectar la presencia de valores ausentes.

En el caso particular de la columna 'chain', observamos que, presenta una baja cantidad de valores ausentes y el tipo de dato asignado no es el adecuado.

Por lo tanto, se procederá a realizar las modificaciones necesarias para garantizar la correcta manipulación y análisis de la información contenida en esta columna 

In [15]:
#Encontrar valores ausentes en la columna 'chain'
missing_values = rest_data['chain'].isnull()
rows_with_missing_values = rest_data[missing_values]
rows_with_missing_values

,id,object_name,address,chain,object_type,number
7408,19194,TAQUERIA LOS 3 CARNALES,5000 E WHITTIER BLVD,NaN,Restaurant,14
7523,19309,JAMMIN JIMMY'S PIZZA,1641 FIRESTONE BLVD,NaN,Pizza,1
8648,20434,THE LEXINGTON THEATER,129 E 3RD ST,NaN,Restaurant,35


In [42]:
filtered_data= rest_data[rest_data['object_name']== "TWO BOOTS PIZZA" ].sum()
filtered_data

id                       12507
object_name    TWO BOOTS PIZZA
address         828 S BROADWAY
chain                    False
object_type              Pizza
number                      19
dtype: object

In [5]:
# Cambio de tipo de dato en la columna 'chain'
rest_data['chain'] = rest_data['chain'].astype(bool)

rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9651 non-null   bool  
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 386.5+ KB


Con los cambios 